In [ ]:
#Importing the required libraries

import  tweepy
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from textblob import TextBlob
import re
import string
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

#Accessing the twitter API keys

CONSUMER_KEY = "4l1xxxxxxxxxxxxxxxKTFW3"
CONSUMER_SECRET = "uvdSMxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxvG3L1"
ACCESS_TOKEN = "xxxxxxxxxxxx873143296-z7qkGxxxxxxxxfvxFIFAGTm8hDwh"
ACCESS_SECRET = "kCYxxxxxxxxxxx6zNIAfnBkHphrEdKpKTQVyxxxxxxG"

#importing access keys

def twitter_setup():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tweepy.API(auth)
    return api

# Extract 100 tweets from the twitter user suing twitter API's(Use username, name, hashtag as required)

extractor = twitter_setup()
posts = extractor.user_timeline(screen_name="@realDonaldTrump", count = 100, lang ="en",tweet_mode="extended")

#  Print 5 recent tweets

print("no. of tweets extracted: {}.\n".format(len(posts)))
print("Show the 5 recent tweets:\n")
i=1
for tweet in posts[:5]:
    print(str(i) +') '+ tweet.full_text + '\n')
    i= i+1
    
#Creating pandas DataFrame

data = pd.DataFrame(data=[tweet.full_text for tweet in posts], columns=['tweets'])
display(data.head(10))

# Adding relevant data to the dataframe

data['len'] = np.array([len(tweet.full_text) for tweet in posts])
data['ID'] = np.array([tweet.id for tweet in posts])
data['Date'] = np.array([tweet.created_at for tweet in posts])
data['Source'] = np.array([tweet.source for tweet in posts])
data['Likes'] = np.array([tweet.favorite_count for tweet in posts])
data['Re_tweets'] = np.array([tweet.retweet_count for tweet in posts])
data.head() 

# We extract the tweet with more LIKEs and more RE_TWEETS

likes_max = np.max(data['Likes'])
re_tweets_max = np.max(data['Re_tweets'])
likes = data[data.Likes == likes_max].index[0]
re_tweets = data[data.Re_tweets == re_tweets_max].index[0]

# Max LIKES
print("The tweet with more likes is: \n{}".format(data['tweets'][likes]))
print("Number of likes: {}".format(likes_max))
print("{} character \n".format(data['len'][likes]))

# Max RE_TWEETS
print("The tweet with more re_tweets is: \n{}".format(data['tweets'][re_tweets]))
print("Number of retweets: {}".format(re_tweets_max))
print("{} character.\n".format(data['len'][re_tweets]))

#Plotting Graph based on tweets and it's length

tweet_len = pd.Series(data = data['len'].values)
tweet_likes = pd.Series(data = data['Likes'].values)   #, index=data['Data'])
tweet_retweets = pd.Series(data = data['Re_tweets'].values) #, index=data['Data']
x = data.index
y = data['len']
plt.xlabel("tweet number")
plt.ylabel("length of tweet")
plt.rcParams["figure.figsize"] = (10,8)
plt.plot(x, y, 'go--')
plt.show()

# Likes and retweets visualization

tweet_likes.plot(figsize=(20,4), label='Likes', legend=True)
tweet_retweets.plot(figsize=(20,4), label='Re_tweets', legend=True)

# We obtain creation of conent sources:

sources = []
for source in data['Source']:
    if source not in sources:
        sources.append(source)
# We print sources list:
print("Creation of content sources:")
for source in sources:
    print("* {}".format(source))
    
# Plotting the usage of sources:
percent = np.zeros(len(sources))
 
for source in data['Source']:
    for index in range(len(sources)):
        if source == sources[index]:
            percent[index] += 1
            pass
 
percent /= 100
# Pie chart:
pie_chart = pd.Series(percent, index=sources, name='Sources')
pie_chart.plot.pie(fontsize=11, autopct='%.2f', figsize=(6, 6))

#Removing data otherthan Alphabets

def remove_pattern(data, pattern):
    r = re.findall(pattern, data)
    for i in r:
        data = re.sub(i, '', data)   
    return data   
# remove twitter handles (@user)
data['cleaned_tweets'] = np.vectorize(remove_pattern)(data['tweets'], "@[\w]*")
#Removing Punctuations, Numbers, and Special Characters
data['cleaned_tweets'] = data['cleaned_tweets'].str.replace("[^a-zA-Z#]", " ")
data['cleaned_tweets'] = data['cleaned_tweets'].str.replace("[https]", " ")
#Removing non-english words
def preprocess2(cleaned_tweets):
    stopword_set = set(stopwords.words("english"))
    return
#Removing Short Words
data['cleaned_tweets'] = data['cleaned_tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data

# Create a function to get the subjectivity and polarity

def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity

# Create two new columns 'Subjectivity' & 'Polarity'
data['Subjectivity'] = data['tweets'].apply(getSubjectivity)
data['Polarity'] = data['tweets'].apply(getPolarity)

# Show the new dataframe with columns 'Subjectivity' & 'Polarity'
data

#Tokenization
#Now we will tokenize all the cleaned tweets in our dataset. 

tokenized_tweet = data['cleaned_tweets'].apply(lambda x: x.split())
tokenized_tweet.head()

#Now let’s stitch these tokens back together.

for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

data['cleaned_tweets'] = tokenized_tweet

# Displaying all words of the tweets

all_words = ' '.join([text for text in data['cleaned_tweets']])
from wordcloud import WordCloud
wordcloud = WordCloud(background_color = 'white', width=800, height=500, random_state=21, max_font_size=110).generate(all_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

# non-racist/sexist words
normal_words =' '.join([text for text in data['cleaned_tweets'][data['Polarity'] == 0]])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(normal_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

# Racist words in words
negative_words = ' '.join([text for text in data['cleaned_tweets'][data['Polarity'] == 1]])
wordcloud = WordCloud(background_color = 'white', width=800, height=500, random_state=21, max_font_size=110).generate(negative_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

# Create a function to compute negative (-1), neutral (0) and positive (+1) analysis based on polarity
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
         return 'Positive'
data['Analysis'] = data['Polarity'].apply(getAnalysis)
# Show the dataframe
data


# We construct lists with classified tweets to get the percentages

pos_tweets = [ tweet for index, tweet in enumerate(data['tweets']) if data['Polarity'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(data['tweets']) if data['Polarity'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(data['tweets']) if data['Polarity'][index] < 0]

# We print percentages:

print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(data['tweets'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(data['tweets'])))
print("Percentage de negative tweets: {}%".format(len(neg_tweets)*100/len(data['tweets'])))

# Finding the number of positive,negative and neutral tweets and plot them in bar chart
data['Analysis'].value_counts()

plt.title('Number count', fontsize=20)
plt.xlabel('Type of emotion', fontsize=16)
plt.ylabel('count', fontsize=16)
data['Analysis'].value_counts().plot(kind = 'bar', color='cyan', edgecolor='black')
plt.show()

# To get the emotion of a particular tweet
data.iloc[120,[7, 10]] #120 = row, [7,10] = columns